# 1. New Lock Object
C# 13 introduces a new `System.Threading.Lock` type for mutual exclusion. Historically, an object type was used for locking purposes, but now there’s a dedicated Lock type, which is expected to become the future standard for most locking.

In [1]:
// Before
public class LockExample 
{
    private readonly object _lock = new();

    public void DoStuff() 
    {
        lock (_lock)
        {
           Console.WriteLine("We're inside old lock");
        }          
    }
}

new LockExample().DoStuff();

We're inside old lock


In [1]:
using System.Threading;

// .NET 9
public class LockExample 
{
    private readonly Lock _lock = new();

    public void DoStuff() 
    {
        lock (_lock)
        {
            Console.WriteLine("We're inside .NET 9 lock");
        }          
    }
}

new LockExample().DoStuff();

We're inside .NET 9 lock


# 2. Task.WhenEach
Imagine you have a list of tasks that finish at different intervals. You want to process each one as soon as it completes. `WaitAll()` won’t work here since it waits for all tasks to finish. We can use `Task.WaitAny()` as a workaround to pick off the next one as it completes. C# 13 introduces a more elegant and efficient approach to handle this scenario: `Task.WhenEach`. Check out the example below.

In [ ]:
var tasks = Enumerable.Range(1, 5)
   .Select(async i =>
   {
     await Task.Delay(new Random().Next(1000, 5000));
     return $"Task {i} done";
   })
   .ToList();

var tasks2 = Enumerable.Range(1, 5)
   .Select(async i =>
   {
     await Task.Delay(new Random().Next(1000, 5000));
     return $"Task {i} done";
   })
   .ToList();

Console.WriteLine("Before .NET 9");
while(tasks.Count > 0)
{
   var completedTask = await Task.WhenAny(tasks);
   tasks.Remove(completedTask);
   Console.WriteLine(await completedTask);
}

Console.WriteLine(".NET 9");
await foreach (var completedTask in Task.WhenEach(tasks2))
   Console.WriteLine(await completedTask);

Before .NET 9
Task 5 done
Task 2 done
Task 3 done
Task 1 done
Task 4 done
.NET 9
Task 2 done
Task 4 done
Task 5 done
Task 1 done
Task 3 done


# 3. params Collections
Starting with C# 13, `params` parameters can be of any types supported for collection expressions.

In [ ]:
// Before
static void WriteNumbersCount(params int[] numbers)
   => Console.WriteLine(numbers.Length);

// .NET 9
static void WriteNumbersCount(params ReadOnlySpan<int> numbers) =>
    Console.WriteLine(numbers.Length);

static void WriteNumbersCount(params IEnumerable<int> numbers) =>
    Console.WriteLine(numbers.Count());

static void WriteNumbersCount(params HashSet<int> numbers) =>
    Console.WriteLine(numbers.Count);

- Cleaner code. Significantly reduces the number of calls to `.ToArray()`, `.ToList()`.
- Performance. Calls like `.ToArray()`, `.ToList()` already add extra resource overhead on their own. Also, it now supports passing `Span<>` and `IEnumerable<>`, leveraging more efficient memory usage and lazy execution. Overall, this provides greater flexibility and better performance in scenarios that demand it 🚀.

# 4. Hybrid Cache
The new HybridCache API addresses certain gaps in the existing `IDistributedCache` and `IMemoryCache` APIs, such as the stampede problem, introduces new features and capabilities, and makes caching in .NET more flexible and performant. HybridCache is designed to be a drop-in replacement for most `IDistributedCache` and `IMemoryCache` scenarios.

In [ ]:
#r "nuget: Microsoft.Extensions.Caching.Abstractions"
#r "nuget: Microsoft.Extensions.Http"
#r "nuget: Microsoft.Extensions.Caching.Memory"
#r "nuget: Microsoft.Extensions.Caching.Hybrid"
#r "nuget: Microsoft.Extensions.Hosting"
#r "nuget: Microsoft.Extensions.DependencyInjection.Abstractions"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;
using System.Net.Http.Json;
using System.Text.Json;
using Microsoft.Extensions.Caching.Distributed;
using Microsoft.Extensions.Caching.Hybrid;
using Microsoft.Extensions.Caching.Memory;

public record Post(int UserId, int Id, string Title, string Body);

public class PostsService(
    IHttpClientFactory httpClientFactory,
    IMemoryCache memoryCache,
    IDistributedCache distributedCache,
    HybridCache hybridCache)
{
    public async Task<List<Post>?> GetUserPostsAsync(string userId)
    {
        var cacheKey = $"posts_{userId}";

        // Before (Memory Cache)
        var posts = await memoryCache.GetOrCreateAsync(cacheKey,
            async _ => await GetPostsAsync(userId));

        // Before (Distributed Cache)
        var postsJson = await distributedCache.GetStringAsync(cacheKey);
        if(postsJson is null)
        {
            posts = await GetPostsAsync(userId);
            await distributedCache.SetStringAsync(cacheKey, JsonSerializer.Serialize(posts));
        } else {
            posts = JsonSerializer.Deserialize<List<Post>>(postsJson);
        }

        // .NET 9 Hybrid Cache
        posts = await hybridCache.GetOrCreateAsync(cacheKey,
            async _ => await GetPostsAsync(userId), new HybridCacheEntryOptions() {
                Flags = HybridCacheEntryFlags.DisableLocalCache | // Act as distributed cache
                        HybridCacheEntryFlags.DisableDistributedCache // Act as local cache
            });

        return posts;
    }

    private async Task<List<Post>?> GetPostsAsync(string userId)
    {
        Console.WriteLine("===========Fetching posts from API");
        var url = $"https://jsonplaceholder.typicode.com/posts?userId={userId}";
        var client = httpClientFactory.CreateClient();
        var response = await client.GetAsync(url);
        response.EnsureSuccessStatusCode();
        return await response.Content.ReadFromJsonAsync<List<Post>>();
    }
}

// Setup and Run
var builder = Host.CreateApplicationBuilder();

// Register dependencies
builder.Services.AddHttpClient();
builder.Services.AddMemoryCache();
builder.Services.AddDistributedMemoryCache();
builder.Services.AddHybridCache();

// Register service
builder.Services.AddSingleton<PostsService>();

var app = builder.Build();

// Resolve and use the service
var postsService = app.Services.GetRequiredService<PostsService>();
var posts = await postsService.GetUserPostsAsync("1");

// Output results
Console.WriteLine($"Fetched {posts?.Count ?? 0} posts for user 1");
if (posts is not null)
{
    foreach (var p in posts)
    {
        Console.WriteLine($"[{p.Id}] {p.Title}");
    }
}

Installed Packages Microsoft.Extensions.Caching.Abstractions, 10.0.1 Microsoft.Extensions.Caching.Hybrid, 10.1.0 Microsoft.Extensions.Caching.Memory, 10.0.1 Microsoft.Extensions.Hosting, 10.0.1 Microsoft.Extensions.Http, 10.0.1

Error: System.IO.FileNotFoundException: Could not load file or assembly 'Microsoft.Extensions.DependencyInjection.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60'. The system cannot find the file specified.
File name: 'Microsoft.Extensions.DependencyInjection.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60'
   at Submission#9.<<Initialize>>d__0.MoveNext()
   at System.Runtime.CompilerServices.AsyncMethodBuilderCore.Start[TStateMachine](TStateMachine& stateMachine)
   at Submission#9.<Initialize>()
   at Submission#9.<Factory>(Object[] submissionArray)
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)